In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
import matplotlib.pyplot as plt
import numpy as np
import csv


device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [2]:
xs_0=list()
xs_0_=list()
xs_1=list()
ys_0=list()
data_number= 4
batch_size = 2
epochs = 10000
#100장정도.. 램 가능
with open ('data/Data_narrow.txt', 'rt' ) as r_n:
    target_r = r_n.readline()
    for j in range(data_number): 
        #print(data_number)
        xs_0=list()
        for i in range(10): 
            temp_red=list()
            temp_blue=list()
            temp_green=list()
            
            r=open ('data/'+str(i+10*j)+'_building_r.csv', 'r' )
            rdr=csv.reader(r)
            for target in rdr:
                temp_red.append(target)
                
            r=open ('data/'+str(i+10*j)+'_building_g.csv', 'r' )
            rdr=csv.reader(r)
            for target in rdr:
                temp_green.append(target)
                
            r=open ('data/'+str(i+10*j)+'_building_b.csv', 'r' )
            rdr=csv.reader(r)
            for target in rdr:
                temp_blue.append(target)
                
            temp_red=np.array(temp_red, dtype=np.float32)
            temp_greed=np.array(temp_green, dtype=np.float32)
            temp_blue=np.array(temp_blue, dtype=np.float32)

            xs_1=np.stack((temp_red, temp_green, temp_blue), axis=0)
            xs_1=list(xs_1) 
            xs_0.append(xs_1)
            
            target_r = r_n.readline()
            if i<10:
                A_r,B_r,C_r,D_r,E_r,F_r,G_r,H_r,I_r,J_r,K_r=target_r.split(',')  

                if i==0:
                    pass

                if i==1:
                    ys_0.append([float(E_r)])

                if i>=2 and i<10:

                    ys_0=ys_0[:-1]+[ys_0[-1]+[float(E_r)]]

        xs_0_.append(xs_0)

x_train = np.array(xs_0_, dtype=np.float32)
y_train = np.array(ys_0, dtype=np.float32)
x_train = torch.from_numpy(x_train).cuda()
y_train = torch.from_numpy(y_train).cuda()

xs_0=list()
xs_0_=list()
xs_1=list()
for j in range(data_number): 
    xs_0=list()
    for i in range(10): 
        temp_red=list()
        temp_blue=list()
        temp_green=list()

        r=open ('data/'+str(i+10*j)+'_r.csv', 'r' )
        rdr=csv.reader(r)
        for target in rdr:
            temp_red.append(target)

        r=open ('data/'+str(i+10*j)+'_g.csv', 'r' )
        rdr=csv.reader(r)
        for target in rdr:
            temp_green.append(target)

        r=open ('data/'+str(i+10*j)+'_b.csv', 'r' )
        rdr=csv.reader(r)
        for target in rdr:
            temp_blue.append(target)

        temp_red=np.array(temp_red, dtype=np.float32)
        temp_greed=np.array(temp_green, dtype=np.float32)
        temp_blue=np.array(temp_blue, dtype=np.float32)

        xs_1=np.stack((temp_red, temp_green, temp_blue), axis=0)
        xs_1=list(xs_1) 
        xs_0.append(xs_1)
    
    xs_0_.append(xs_0)
        
x_test = np.array(xs_0_, dtype=np.float32)
x_test = torch.from_numpy(x_test).cuda()
#print(x_train.shape,y_train.shape)
dataset = TensorDataset(x_train, y_train)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

dataset_test = TensorDataset(x_test, y_train)
dataloader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=True)
print(x_test.shape)

xs_0=list()
xs_0_=list()
xs_1=list()
ys_0=list()
temp_red=list()
temp_blue=list()
temp_green=list()

print("data_ready")

torch.Size([4, 10, 3, 481, 793])
data_ready


In [3]:
class LRCN(nn.Module):
    def __init__(self):
        super(LRCN, self).__init__()
        self.layer_1 = nn.Conv2d(3,15,5,stride=2)
        self.layer_2 = nn.Conv2d(15,13,5,stride=2)
        self.layer_3 = nn.Conv2d(13,7,5,stride=2)
        self.layer_4 = nn.Conv2d(7,3,5,stride=2)
        self.layer_5 = nn.Conv2d(3,1,5,stride=2)
        self.layer_6 = nn.LSTM(252,100,2, batch_first = True)
        self.layer_7 = nn.Linear(100, 9)
        
        self.batch_size = batch_size
        self.length = x_train.shape[1]
        self.relu = nn.ReLU()
        
        
    def forward(self, x):
        x = x.reshape((-1, 3, 481, 793))
        x = self.layer_1(x)
        x = self.relu(x)
        x = self.layer_2(x)
        x = self.relu(x)
        x = self.layer_3(x)
        x = self.relu(x)
        x = self.layer_4(x)
        x = self.relu(x)
        x = self.layer_5(x)
        x = self.relu(x)
        x = x.reshape((self.batch_size, self.length, x.shape[-3],x.shape[-2],x.shape[-1]))
        x = x.reshape((self.batch_size, self.length, x.shape[-3]*x.shape[-2]*x.shape[-1]))
        x, (h_n, c_n) = self.layer_6(x)
        x = self.relu(h_n[-1,:,:].reshape((self.batch_size,-1)))
        x = self.layer_7(x)
        return x

model = LRCN().cuda()
loss = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

def train(dataloader, model, loss, optimizer):
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        pred = model(X)
        loss_result = loss(pred, y)
        optimizer.zero_grad()
        loss_result.backward()
        optimizer.step()
            
def test(dataloader, model, loss):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss(pred, y).item()
    test_loss /= num_batches
    print(f"Test Error: \n Avg loss: {test_loss:>8f} \n")

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(dataloader, model, loss, optimizer)
    test(dataloader_test, model, loss)
print("Done!")


Epoch 1
-------------------------------
Test Error: 
 Avg loss: 21.883407 

Epoch 2
-------------------------------
Test Error: 
 Avg loss: 21.860901 

Epoch 3
-------------------------------
Test Error: 
 Avg loss: 21.838424 

Epoch 4
-------------------------------
Test Error: 
 Avg loss: 21.815918 

Epoch 5
-------------------------------
Test Error: 
 Avg loss: 21.793486 

Epoch 6
-------------------------------
Test Error: 
 Avg loss: 21.771049 

Epoch 7
-------------------------------
Test Error: 
 Avg loss: 21.748653 

Epoch 8
-------------------------------
Test Error: 
 Avg loss: 21.726244 

Epoch 9
-------------------------------
Test Error: 
 Avg loss: 21.703905 

Epoch 10
-------------------------------
Test Error: 
 Avg loss: 21.681537 

Epoch 11
-------------------------------
Test Error: 
 Avg loss: 21.659210 

Epoch 12
-------------------------------
Test Error: 
 Avg loss: 21.636911 

Epoch 13
-------------------------------
Test Error: 
 Avg loss: 21.614605 

Epoch 14

Test Error: 
 Avg loss: 19.440214 

Epoch 114
-------------------------------
Test Error: 
 Avg loss: 19.418303 

Epoch 115
-------------------------------
Test Error: 
 Avg loss: 19.396348 

Epoch 116
-------------------------------
Test Error: 
 Avg loss: 19.374426 

Epoch 117
-------------------------------
Test Error: 
 Avg loss: 19.352456 

Epoch 118
-------------------------------
Test Error: 
 Avg loss: 19.330458 

Epoch 119
-------------------------------
Test Error: 
 Avg loss: 19.308442 

Epoch 120
-------------------------------
Test Error: 
 Avg loss: 19.286469 

Epoch 121
-------------------------------
Test Error: 
 Avg loss: 19.264383 

Epoch 122
-------------------------------
Test Error: 
 Avg loss: 19.242282 

Epoch 123
-------------------------------
Test Error: 
 Avg loss: 19.220191 

Epoch 124
-------------------------------
Test Error: 
 Avg loss: 19.198057 

Epoch 125
-------------------------------
Test Error: 
 Avg loss: 19.175884 

Epoch 126
------------------

Test Error: 
 Avg loss: 16.737843 

Epoch 226
-------------------------------
Test Error: 
 Avg loss: 16.709912 

Epoch 227
-------------------------------
Test Error: 
 Avg loss: 16.681950 

Epoch 228
-------------------------------
Test Error: 
 Avg loss: 16.653921 

Epoch 229
-------------------------------
Test Error: 
 Avg loss: 16.625779 

Epoch 230
-------------------------------
Test Error: 
 Avg loss: 16.597556 

Epoch 231
-------------------------------
Test Error: 
 Avg loss: 16.569171 

Epoch 232
-------------------------------
Test Error: 
 Avg loss: 16.540681 

Epoch 233
-------------------------------
Test Error: 
 Avg loss: 16.512135 

Epoch 234
-------------------------------
Test Error: 
 Avg loss: 16.483565 

Epoch 235
-------------------------------
Test Error: 
 Avg loss: 16.454943 

Epoch 236
-------------------------------
Test Error: 
 Avg loss: 16.425902 

Epoch 237
-------------------------------
Test Error: 
 Avg loss: 16.397110 

Epoch 238
------------------

Test Error: 
 Avg loss: 12.798136 

Epoch 338
-------------------------------
Test Error: 
 Avg loss: 12.753357 

Epoch 339
-------------------------------
Test Error: 
 Avg loss: 12.708014 

Epoch 340
-------------------------------
Test Error: 
 Avg loss: 12.662686 

Epoch 341
-------------------------------
Test Error: 
 Avg loss: 12.617029 

Epoch 342
-------------------------------
Test Error: 
 Avg loss: 12.571401 

Epoch 343
-------------------------------
Test Error: 
 Avg loss: 12.525475 

Epoch 344
-------------------------------
Test Error: 
 Avg loss: 12.479484 

Epoch 345
-------------------------------
Test Error: 
 Avg loss: 12.433064 

Epoch 346
-------------------------------
Test Error: 
 Avg loss: 12.386647 

Epoch 347
-------------------------------
Test Error: 
 Avg loss: 12.339595 

Epoch 348
-------------------------------
Test Error: 
 Avg loss: 12.292792 

Epoch 349
-------------------------------
Test Error: 
 Avg loss: 12.245555 

Epoch 350
------------------

Test Error: 
 Avg loss: 6.728679 

Epoch 448
-------------------------------
Test Error: 
 Avg loss: 6.667498 

Epoch 449
-------------------------------
Test Error: 
 Avg loss: 6.606853 

Epoch 450
-------------------------------
Test Error: 
 Avg loss: 6.545611 

Epoch 451
-------------------------------
Test Error: 
 Avg loss: 6.485043 

Epoch 452
-------------------------------
Test Error: 
 Avg loss: 6.424069 

Epoch 453
-------------------------------
Test Error: 
 Avg loss: 6.363323 

Epoch 454
-------------------------------
Test Error: 
 Avg loss: 6.302881 

Epoch 455
-------------------------------
Test Error: 
 Avg loss: 6.242326 

Epoch 456
-------------------------------
Test Error: 
 Avg loss: 6.182226 

Epoch 457
-------------------------------
Test Error: 
 Avg loss: 6.121523 

Epoch 458
-------------------------------
Test Error: 
 Avg loss: 6.061294 

Epoch 459
-------------------------------
Test Error: 
 Avg loss: 6.001026 

Epoch 460
-------------------------------

...

Test Error: 
 Avg loss: 0.589070 

Epoch 895
-------------------------------
Test Error: 
 Avg loss: 0.589065 

Epoch 896
-------------------------------
Test Error: 
 Avg loss: 0.589035 

Epoch 897
-------------------------------
Test Error: 
 Avg loss: 0.589055 

Epoch 898
-------------------------------
Test Error: 
 Avg loss: 0.588987 

Epoch 899
-------------------------------
Test Error: 
 Avg loss: 0.589039 

Epoch 900
-------------------------------
Test Error: 
 Avg loss: 0.589092 

Epoch 901
-------------------------------
Test Error: 
 Avg loss: 0.589088 

Epoch 902
-------------------------------
Test Error: 
 Avg loss: 0.589099 

Epoch 903
-------------------------------
Test Error: 
 Avg loss: 0.589056 

Epoch 904
-------------------------------
Test Error: 
 Avg loss: 0.589088 

Epoch 905
-------------------------------
Test Error: 
 Avg loss: 0.589050 

Epoch 906
-------------------------------
Test Error: 
 Avg loss: 0.589012 

Epoch 907
-------------------------------

Test Error: 
 Avg loss: 0.588335 

Epoch 1006
-------------------------------
Test Error: 
 Avg loss: 0.588351 

Epoch 1007
-------------------------------
Test Error: 
 Avg loss: 0.588348 

Epoch 1008
-------------------------------
Test Error: 
 Avg loss: 0.588368 

Epoch 1009
-------------------------------
Test Error: 
 Avg loss: 0.588356 

Epoch 1010
-------------------------------
Test Error: 
 Avg loss: 0.588347 

Epoch 1011
-------------------------------
Test Error: 
 Avg loss: 0.588344 

Epoch 1012
-------------------------------
Test Error: 
 Avg loss: 0.588351 

Epoch 1013
-------------------------------
Test Error: 
 Avg loss: 0.588338 

Epoch 1014
-------------------------------
Test Error: 
 Avg loss: 0.588333 

Epoch 1015
-------------------------------
Test Error: 
 Avg loss: 0.588330 

Epoch 1016
-------------------------------
Test Error: 
 Avg loss: 0.588339 

Epoch 1017
-------------------------------
Test Error: 
 Avg loss: 0.588286 

Epoch 1018
------------------

Test Error: 
 Avg loss: 0.587919 

Epoch 1114
-------------------------------
Test Error: 
 Avg loss: 0.587922 

Epoch 1115
-------------------------------
Test Error: 
 Avg loss: 0.587888 

Epoch 1116
-------------------------------
Test Error: 
 Avg loss: 0.587927 

Epoch 1117
-------------------------------
Test Error: 
 Avg loss: 0.587916 

Epoch 1118
-------------------------------
Test Error: 
 Avg loss: 0.587914 

Epoch 1119
-------------------------------
Test Error: 
 Avg loss: 0.587930 

Epoch 1120
-------------------------------
Test Error: 
 Avg loss: 0.587911 

Epoch 1121
-------------------------------
Test Error: 
 Avg loss: 0.587928 

Epoch 1122
-------------------------------
Test Error: 
 Avg loss: 0.587919 

Epoch 1123
-------------------------------
Test Error: 
 Avg loss: 0.587930 

Epoch 1124
-------------------------------
Test Error: 
 Avg loss: 0.587905 

Epoch 1125
-------------------------------
Test Error: 
 Avg loss: 0.587892 

Epoch 1126
------------------

Test Error: 
 Avg loss: 0.587379 

Epoch 1222
-------------------------------
Test Error: 
 Avg loss: 0.587297 

Epoch 1223
-------------------------------
Test Error: 
 Avg loss: 0.587380 

Epoch 1224
-------------------------------
Test Error: 
 Avg loss: 0.587358 

Epoch 1225
-------------------------------
Test Error: 
 Avg loss: 0.587381 

Epoch 1226
-------------------------------
Test Error: 
 Avg loss: 0.587327 

Epoch 1227
-------------------------------
Test Error: 
 Avg loss: 0.587251 

Epoch 1228
-------------------------------
Test Error: 
 Avg loss: 0.587288 

Epoch 1229
-------------------------------
Test Error: 
 Avg loss: 0.587282 

Epoch 1230
-------------------------------
Test Error: 
 Avg loss: 0.587257 

Epoch 1231
-------------------------------
Test Error: 
 Avg loss: 0.587323 

Epoch 1232
-------------------------------
Test Error: 
 Avg loss: 0.587256 

Epoch 1233
-------------------------------
Test Error: 
 Avg loss: 0.587241 

Epoch 1234
------------------

KeyboardInterrupt: 

In [ ]:
import torch
import cv2
import numpy as np
from torchsummary import summary
from torch.nn import functional as F

class ModelOutputs_resnet():
    def __init__(self, model, target_layers, target_sub_layers):
        self.model = model
        self.target_layers = target_layers
        self.target_sub_layers = target_sub_layers
        self.gradients = []

    def save_gradient(self, grad):
        self.gradients.append(grad)

    def get_gradients(self):
        return self.gradients

    def __call__(self, x):
        self.gradients = []
        for name, module in self.model.named_children(): # 모든 layer에 대해서 직접 접근
            x = module(x)
            if name== 'avgpool': # avgpool이후 fully connect하기 전 data shape을 flatten시킴
                x = torch.flatten(x,1)
            if name in self.target_layers: # target_layer라면 해당 layer에서의 gradient를 저장
                for sub_name, sub_module in module[len(module)-1].named_children():
                    if sub_name in self.target_sub_layers:
                        x.register_hook(self.save_gradient) #
                        target_feature_maps = x # x's shape = 512X14X14(C,W,H) feature map
        return target_feature_maps, x # target_activation : target_activation_layer's feature maps // output : classification ( ImageNet's classes : 1000 )


class GradCam_resnet:
    def __init__(self, model, target_layer_names,target_sub_layer_names, use_cuda):
        self.model = model
        self.model.eval()
        self.cuda = use_cuda
        if self.cuda:  # GPU일 경우 model을 cuda로 설정
            self.model = model.cuda()

        self.extractor = ModelOutputs_resnet(self.model, target_layer_names,target_sub_layer_names)

    def forward(self, input):
        return self.model(input)

    def __call__(self, input, index=None):

        if self.cuda:  # GPU일 경우 input을 cuda로 변환하여 전달
            features, output = self.extractor(input.cuda())
        else:
            features, output = self.extractor(input)

        probs,idx = 0, 0
        if index == None:
            index = np.argmax(output.cpu().data.numpy())  # index = 정답이라고 추측한 class index
            h_x = F.softmax(output,dim=1).data.squeeze()
            probs, idx = h_x.sort(0,True)


        one_hot = np.zeros((1, output.size()[-1]), dtype=np.float32)
        one_hot[0][index] = 1  # 정답이라고 생각하는 class의 index 리스트 위치의 값만 1로
        one_hot = torch.from_numpy(one_hot).requires_grad_(True)  # numpy배열을 tensor로 변환
        # requires_grad == True 텐서의 모든 연산에 대하여 추적
        if self.cuda:
            one_hot = torch.sum(one_hot.cuda() * output)
        else:
            one_hot = torch.sum(one_hot * output)

        self.model.zero_grad()
        one_hot.backward(retain_graph=True)

        grads_val = self.extractor.get_gradients()[-1].cpu().data.numpy()

        target = features  # A^k

        target_cam = target.cpu().data.numpy()
        bz, nc, h,w = target_cam.shape

        target = target.cpu().data.numpy()[0, :]

        params = list(self.model.parameters())

        weight_softmax = np.squeeze(params[-2].data.cpu().numpy())

        cam = weight_softmax[index].dot(target_cam.reshape((nc,h*w)))
        cam = cam.reshape(h,w)
        cam = np.maximum(cam, 0)
        cam = cv2.resize(cam, (224, 224))  # 224X224크기로 변환
        cam = cam - np.min(cam)
        cam = cam / np.max(cam)


        weights = np.mean(grads_val, axis=(2, 3))[0, :]  # 논문에서의 global average pooling 식에 해당하는 부분
        grad_cam = np.zeros(target.shape[1:], dtype=np.float32)  # 14X14

        for i, w in enumerate(weights): # calcul grad_cam
            grad_cam += w * target[i, :, :]  # linear combination L^c_{Grad-CAM}에 해당하는 식에서 ReLU를 제외한 식

        grad_cam = np.maximum(grad_cam, 0)  # 0보다 작은 값을 제거
        grad_cam = cv2.resize(grad_cam, (224, 224))  # 224X224크기로 변환
        grad_cam = grad_cam - np.min(grad_cam)  #
        grad_cam = grad_cam / np.max(grad_cam)  # 위의 것과 해당 줄의 것은 0~1사이의 값으로 정규화하기 위한 정리
        return grad_cam, cam ,index,probs,idx ,

In [ ]:
import torch
import cv2
import numpy as np



from torch.nn import functional as F


class FeatureExtractor_vgg():
    """ Class for extracting activations and
    registering gradients from targetted intermediate layers """

    def __init__(self, model, target_layers): # target_layers = 35 ==> VGG19에서 가장 마지막 MaxPool2D전 ReLU함수
        self.model = model
        self.target_layers = target_layers
        self.gradients = []

    def save_gradient(self, grad):
        self.gradients.append(grad)


    def __call__(self, x):
        self.gradients = []
        for name, module in self.model._modules.items(): # 모든 layer에 대해서 직접 접근
            x = module(x)
            if name in self.target_layers: # target_layer라면 해당 layer에서의 gradient를 저장
                x.register_hook(self.save_gradient) #
                target_feature_maps = x # x's shape = 512X14X14(C,W,H) feature map
        return target_feature_maps, x


class ModelOutputs_vgg():
    """ Class for making a forward pass, and getting:
    1. The network output.
    2. Activations from intermeddiate targetted layers.
    3. Gradients from intermeddiate targetted layers. """

    def __init__(self, model, target_layers):
        self.model = model
        self.feature_extractor = FeatureExtractor_vgg(self.model.features, target_layers)

    def get_gradients(self):
        return self.feature_extractor.gradients

    def __call__(self, x):
        target_activations, output = self.feature_extractor(x)
        output = output.view(output.size(0), -1)
        output = self.model.classifier(output) # feature extract를 통해서 나온 값을 활용하여 classification 진행
        #print("ModelOutputs().output.shape : ",output[0])
        #print("ModelOutputs().target_activations.shape :",target_activations[0])
        return target_activations, output

class GradCam_vgg:
    def __init__(self, model, target_layer_names, use_cuda):
        self.model = model
        self.model.eval()
        self.cuda = use_cuda
        if self.cuda: # GPU일 경우 model을 cuda로 설정
            self.model = model.cuda()

        self.extractor = ModelOutputs_vgg(self.model, target_layer_names)

    def forward(self, input):
        return self.model(input)

    def __call__(self, input, index=None):
        if self.cuda: # GPU일 경우 input을 cuda로 변환하여 전달
            features, output = self.extractor(input.cuda())
        else:
            features, output = self.extractor(input)
        #print("features : ",features.cpu().data.numpy().shape) # 해당 위치에서 추출된 feature map ( 512,14,14 ) (ChannelX14X14)
        #print("output : ",output.cpu().data.numpy().shape) # class를 의미함
        probs, idx = 0,0
        #print("index : ", index)
        if index == None:
            index = np.argmax(output.cpu().data.numpy())  # index = 정답이라고 추측한 class index
            h_x = F.softmax(output,dim=1).data.squeeze()
            probs, idx = h_x.sort(0,True)
        #print("index : ", index)
        one_hot = np.zeros((1, output.size()[-1]), dtype=np.float32)
        one_hot[0][index] = 1 # 정답이라고 생각하는 class의 index 리스트 위치의 값만 1로
        one_hot = torch.from_numpy(one_hot).requires_grad_(True) # numpy배열을 tensor로 변환
        # requires_grad == True 텐서의 모든 연산에 대하여 추적
        if self.cuda:
            one_hot = torch.sum(one_hot.cuda() * output)
        else:
            one_hot = torch.sum(one_hot * output)

        self.model.features.zero_grad()
        self.model.classifier.zero_grad()
        one_hot.backward(retain_graph=True)

        grads_val = self.extractor.get_gradients()[-1].cpu().data.numpy()
        #print("grads_val : ",grads_val.shape) # 512 X 14 X 14
        target = features  # A^k
        target = target.cpu().data.numpy()[0, :]

        cam = None

        weights = np.mean(grads_val, axis=(2, 3))[0, :]  # 논문에서의 global average pooling 식에 해당하는 부분
        grad_cam = np.zeros(target.shape[1:], dtype=np.float32)  # 14X14

        for i, w in enumerate(weights):  # calcul grad_cam
            grad_cam += w * target[i, :, :]  # linear combination L^c_{Grad-CAM}에 해당하는 식에서 ReLU를 제외한 식

        grad_cam = np.maximum(grad_cam, 0)  # 0보다 작은 값을 제거
        grad_cam = cv2.resize(grad_cam, (224, 224))  # 224X224크기로 변환
        grad_cam = grad_cam - np.min(grad_cam)  #
        grad_cam = grad_cam / np.max(grad_cam)  # 위의 것과 해당 줄의 것은 0~1사이의 값으로 정규화하기 위한 정리
        return grad_cam, cam, index, probs, idx,